![pipeline](pictures/pictures.002.png)

# Text-Fabric from ETCBC

This notebook assembles the data from the ETCBC that is needed
to compile its datasets in text-fabric-format on Github.
Ulltimately the data for the website [SHEBANQ](https://shebanq.ancient-data.org) will be
derived from these TF-sources.

## Pipeline
This is **pipe 1** of the pipeline from ETCBC data to the website SHEBANQ.

A run of this pipe produces a data *version*.
It should be run whenever there are new or updated data sources present that affect the output data.
Since all input data is delivered in a Github repo, we have excellent machinery to 
work with versioning.

The pipe works by executing a series of programs, contained in Github repositories.
For each repository in the pipe, a series of notebooks will be executed.
See [script mode](https://github.com/ETCBC/pipeline/blob/master/README.md#operation) for 
details on how we call notebooks.

All this is specified in the configuration below.

### Core data

The core data is delivered by the ETCBC as `bhsa.mql.bz2` in 
the Github repo [bhsa](https://github.com/ETCBC/bhsa) in directory `source`.

This data will be converted by `tfFromMQL` in the `programs` directory.

The result of this action will be an updated TF resource in its 
`tf/core` directory.

### Additional data

Researchers have contributed to the dataset, 
but not all that data is in the core.
They are typically in the repository where the research has been 
executed, and where the data is documented.

Before the pipe starts, these repos must be pulled.

## Continuous version
Version `c` acts as a *continuous* version. It will be overwritten
by new snapshots of the data on a regular basis.

We support the following workflow to carry out these updates:

1. make a new version called `_temp`. Note:
   * this choice of name prevents it to reach github, because `_temp` directories are in `.gitignore`;
   * after running this workflow, the version `_temp` already exists, this is not a problem;
2. put a new data snapshot in the `source/_temp` directory of the `bhsa` repo, add also data to the
   `source/_temp` directories of the other repos in the pipeline, as far as relevant;
3. run `good = runPipeline(pipeline, versions=['_temp'], force=True)`. Note:
   * we use `force=True` here, because then the old data in version `_temp` will be thoroughly overwritten;
4. if all went well run `copyVersion(pipeline, '_temp', 'c')`. This will overwrite all data directories
   in version `c` by the just created data directories in `temp`.

In [1]:
import os,sys,collections
from pipeline import runPipeline, copyVersion
from tf.fabric import Fabric

# Config

In [2]:
CORE_NAME = 'bhsa'

if 'SCRIPT' not in locals(): 
    SCRIPT = False
    DEFAULT_CORE_NAME = CORE_NAME
    DEFAULT_VERSION = 'c'

# Pipeline settings

Here all the nitty gritty differences between versions are stated.

In [3]:
pipeline = dict(
    defaults = dict(
        CORE_NAME=CORE_NAME,
        VERSION=DEFAULT_VERSION,
        LANG_FEATURE='language',
        OCC_FEATURE='g_cons',
        LEX_FEATURE='lex',
        TEXT_FEATURE='g_word_utf8',
        TRAILER_FEATURE='trailer_utf8',
        DO_VOCALIZED_LEXEME=True,
        EXTRA_OVERLAP='',
        LEX_FORMATS='@fmt:lex-trans-plain={lex0} ',
        RENAME=(
            ('g_suffix', 'trailer'),
            ('g_suffix_utf8', 'trailer_utf8'),
        ),
    ),
    versions={
        '_temp': dict(),
        'c': dict(),
        '2017': dict(),
        '2016': dict(),
        '4b': dict(
                DO_VOCALIZED_LEXEME=False,
                EXTRA_OVERLAP='gloss nametype',
                LEX_FORMATS='@fmt:lex-trans-plain={lex} ',
            ),
        '4': dict(
                DO_VOCALIZED_LEXEME=False,
                EXTRA_OVERLAP='gloss nametype',
                LEX_FORMATS='@fmt:lex-trans-plain={lex} ',
        ),
        '3': dict(
                LANG_FEATURE='language',
                OCC_FEATURE='surface_consonants',
                LEX_FEATURE='lexeme',
                TEXT_FEATURE='text',
                TRAILER_FEATURE='suffix',
                LEX_FORMAT='@fmt:lex-trans-plain={lexeme} ',
            ),
    },
    repoOrder = '''
        bhsa
        phono
        valence
        parallels
    ''',
    repoConfig = dict(
        bhsa=(
            dict(
                task='coreData',
            ),
            dict(
                task='bookNames',
                omit={},
            ),
            dict(
                task='lexicon',
                omit={'3'},
            ),
            dict(
                task='paragraphs',
                 omit={'3', '4', '4b'},
            ),
            dict(
                task='ketivQere',
                omit={'3', '4', '4b'},
            ),
            dict(
                task='stats',
                omit={'4', '4b'},
            ),
        ),
        phono=(
            dict(
                task='phono',
                omit={'3', '4', '4b'},
            ),
        ),
        valence=(
            dict(
                task='enrich',
                omit={'3'},
            ),
            dict(
                task='flowchart',
                omit={'3'},
            ),
        ),
        parallels=(
            dict(
                task='parallels',
                omit={},
                params=dict(
                    FORCE_MATRIX=False,
                ),
            ),
        ),
    ),
    repoDataDirs = dict(
        bhsa      = 'source _temp tf shebanq',
        phono     = '_temp tf',
        valence   = 'source _temp tf shebanq',
        parallels = 'source _temp tf',
    ),
)

# Run the pipeline

To run everything from scratch:

```python
good = runPipeline(pipeline, versions=['3', '4', '4b', '2016', '2017', 'c'], force=True)
```

To make a new version called `temp`

```python
good = runPipeline(pipeline, versions=['_temp'], force=True) # a new candidate for 'c'
```

To copy the `_temp` version over to the continuous version `c`:

```python
if good:
    copyVersion(pipeline, '_temp', 'c')
```
This will copy all data, `source`, `_temp` and `tf` over from `_temp` to `c`.

If you want to create the node mappings between versions,
go to the versionMappings notebook in the BHSA repo and run it.

In [4]:
good = runPipeline(pipeline, versions=['3', '4', '4b', '2016', '2017', 'c'], force=True)


##############################################################################################
#                                                                                            #
#       0.00s Make version [3]                                                               #
#                                                                                            #
##############################################################################################


**********************************************************************************************
*                                                                                            *
*       0.00s Make repo [bhsa]                                                               *
*                                                                                            *
**********************************************************************************************


---------------------------------------------

  0.19s 		otype phrase_atom
  0.19s 			feature parents (str) =def= id_d : node
  0.19s 			feature phrase_atom_relation (str) =def= Appo : node
  0.19s 			feature mother (str) =def= 0 : edge
  0.20s 			feature determination (str) =def= NA : node
  0.20s 			feature is_apposition (str) =def= false : node
  0.20s 			feature phrase_atom_type (str) =def= VP : node
  0.20s 			feature phrase_atom_number (int) =def= 0 : node
  0.20s 		otype phrase
  0.20s 			feature parents (str) =def= id_d : node
  0.20s 			feature phrase_function (str) =def= none : node
  0.20s 			feature determination (str) =def= NA : node
  0.21s 			feature is_apposition (str) =def= false : node
  0.21s 			feature phrase_type (str) =def= VP : node
  0.21s 			feature number_within_clause (int) =def= 0 : node
  0.21s 		otype subphrase
  0.21s 			feature parents (str) =def= id_d : node
  0.21s 			feature mother (str) =def= 0 : edge
  0.22s 			feature subphrase_kind (str) =def= mother : node
  0.22s 			feature subphrase_type (s

   |     0.68s T graphical_root_formation_plain to /Users/dirk/github/etcbc/bhsa/_temp/3/tf
   |     0.67s T graphical_verbal_ending to /Users/dirk/github/etcbc/bhsa/_temp/3/tf
   |     0.67s T graphical_verbal_ending_plain to /Users/dirk/github/etcbc/bhsa/_temp/3/tf
   |     0.76s T graphical_word       to /Users/dirk/github/etcbc/bhsa/_temp/3/tf
   |     0.08s T half_verse           to /Users/dirk/github/etcbc/bhsa/_temp/3/tf
   |     0.15s T indentation          to /Users/dirk/github/etcbc/bhsa/_temp/3/tf
   |     0.86s T is_apposition        to /Users/dirk/github/etcbc/bhsa/_temp/3/tf
   |     0.74s T language             to /Users/dirk/github/etcbc/bhsa/_temp/3/tf
   |     0.17s T levels_of_embedding  to /Users/dirk/github/etcbc/bhsa/_temp/3/tf
   |     0.75s T lexeme               to /Users/dirk/github/etcbc/bhsa/_temp/3/tf
   |     0.95s T lexeme_utf8          to /Users/dirk/github/etcbc/bhsa/_temp/3/tf
   |     0.87s T lexical_set          to /Users/dirk/github/etcbc/bhsa/_temp

   |     0.76s T graphical_nominal_ending_plain from /Users/dirk/github/etcbc/bhsa/tf/3
   |     0.71s T graphical_preformative from /Users/dirk/github/etcbc/bhsa/tf/3
   |     0.70s T graphical_preformative_plain from /Users/dirk/github/etcbc/bhsa/tf/3
   |     0.75s T graphical_pron_suffix from /Users/dirk/github/etcbc/bhsa/tf/3
   |     0.72s T graphical_pron_suffix_plain from /Users/dirk/github/etcbc/bhsa/tf/3
   |     0.67s T graphical_root_formation from /Users/dirk/github/etcbc/bhsa/tf/3
   |     0.65s T graphical_root_formation_plain from /Users/dirk/github/etcbc/bhsa/tf/3
   |     0.68s T graphical_verbal_ending from /Users/dirk/github/etcbc/bhsa/tf/3
   |     0.68s T graphical_verbal_ending_plain from /Users/dirk/github/etcbc/bhsa/tf/3
   |     0.13s T half_verse           from /Users/dirk/github/etcbc/bhsa/tf/3
   |     0.17s T indentation          from /Users/dirk/github/etcbc/bhsa/tf/3
   |     1.74s T is_apposition        from /Users/dirk/github/etcbc/bhsa/tf/3
   |     1

   |     0.00s T book@ar              to /Users/dirk/github/etcbc/bhsa/_temp/3/tf
   |     0.00s T book@bn              to /Users/dirk/github/etcbc/bhsa/_temp/3/tf
   |     0.00s T book@da              to /Users/dirk/github/etcbc/bhsa/_temp/3/tf
   |     0.00s T book@de              to /Users/dirk/github/etcbc/bhsa/_temp/3/tf
   |     0.00s T book@el              to /Users/dirk/github/etcbc/bhsa/_temp/3/tf
   |     0.00s T book@en              to /Users/dirk/github/etcbc/bhsa/_temp/3/tf
   |     0.00s T book@es              to /Users/dirk/github/etcbc/bhsa/_temp/3/tf
   |     0.00s T book@fa              to /Users/dirk/github/etcbc/bhsa/_temp/3/tf
   |     0.00s T book@fr              to /Users/dirk/github/etcbc/bhsa/_temp/3/tf
   |     0.00s T book@he              to /Users/dirk/github/etcbc/bhsa/_temp/3/tf
   |     0.00s T book@hi              to /Users/dirk/github/etcbc/bhsa/_temp/3/tf
   |     0.00s T book@id              to /Users/dirk/github/etcbc/bhsa/_temp/3/tf
   |     0.00s T

...

|  2h 04m 19s 	Destination /Users/dirk/github/etcbc/valence/tf/c/.tf/valence.tfx exists
|  2h 04m 19s NOTE: repo seems up to date. Will be run because of "force=True"
True True
..............................................................................................
.  2h 04m 19s Load the existing TF dataset                                                   .
..............................................................................................
This is Text-Fabric 3.0.9
Api reference : https://github.com/Dans-labs/text-fabric/wiki/Api
Tutorial      : https://github.com/Dans-labs/text-fabric/blob/master/docs/tutorial.ipynb
Example data  : https://github.com/Dans-labs/text-fabric-data

113 features found and 0 ignored
  0.00s loading features ...
   |     0.23s B lex_utf8             from /Users/dirk/github/etcbc/bhsa/tf/c
   |     0.17s B lex                  from /Users/dirk/github/etcbc/bhsa/tf/c
   |     0.01s B gloss                from /Users/dirk/github/etcbc/bhsa/tf/

|  2h 05m 06s 	Done
|  2h 05m 06s 	Phrases of kind C :  19302
|  2h 05m 06s 	Phrases of kind L :  11680
|  2h 05m 06s 	Phrases of kind I :   6018
|  2h 05m 06s 	Total complements :  37000
|  2h 05m 06s 	Total phrases     : 214541
..............................................................................................
.  2h 05m 06s Checking enrichment logic                                                      .
..............................................................................................
|  2h 05m 06s 	All 6 rules OK
..............................................................................................
.  2h 05m 06s Generating enrichments                                                         .
..............................................................................................
|  2h 05m 12s 	Generated enrichment values for 1380 verbs:
|  2h 05m 12s 	Enriched values for 221366 nodes
|  2h 05m 12s 	Overview of rule applications:
|  2h 05m 12s gen

   |     0.19s B typ                  from /Users/dirk/github/etcbc/bhsa/tf/c
   |     0.07s B function             from /Users/dirk/github/etcbc/bhsa/tf/c
   |     0.77s T valence              from /Users/dirk/github/etcbc/valence/tf/c
   |     0.79s T predication          from /Users/dirk/github/etcbc/valence/tf/c
   |     0.76s T grammatical          from /Users/dirk/github/etcbc/valence/tf/c
   |     0.36s T original             from /Users/dirk/github/etcbc/valence/tf/c
   |     0.50s T lexical              from /Users/dirk/github/etcbc/valence/tf/c
   |     0.49s T semantic             from /Users/dirk/github/etcbc/valence/tf/c
   |     0.36s T f_correction         from /Users/dirk/github/etcbc/valence/tf/c
   |     0.38s T s_manual             from /Users/dirk/github/etcbc/valence/tf/c
   |     0.48s T cfunction            from /Users/dirk/github/etcbc/valence/tf/c
   |     0.00s Feature overview: 118 for nodes; 4 for edges; 1 configs; 7 computed
    10s All features loaded/comp

|  2h 05m 56s 	10000 clauses
|  2h 06m 00s 	20000 clauses
|  2h 06m 03s 	30000 clauses
|  2h 06m 06s 	40000 clauses
|  2h 06m 08s 	47385 clauses
..............................................................................................
.  2h 06m 08s Writing sense feature to TF                                                    .
..............................................................................................
   |     0.10s T sense                to /Users/dirk/github/etcbc/valence/_temp/c/tf
..............................................................................................
.  2h 06m 09s Check differences with previous version                                        .
..............................................................................................
|  2h 06m 09s 	no features to add
|  2h 06m 09s 	no features to delete
|  2h 06m 09s 	1 features in common
|  2h 06m 09s sense                     ... no changes
|  2h 06m 09s Done
...................

116 features found and 0 ignored
  0.00s loading features ...
   |     0.07s T crossref             from /Users/dirk/github/etcbc/parallels/tf/c
   |     0.04s T crossrefSET          from /Users/dirk/github/etcbc/parallels/tf/c
   |     0.07s T crossrefLCS          from /Users/dirk/github/etcbc/parallels/tf/c
   |     0.00s Feature overview: 108 for nodes; 7 for edges; 1 configs; 7 computed
  4.94s All features loaded/computed - for details use loadLog()
..............................................................................................
.  2h 06m 30s Test: crossrefs of Genesis 10                                                  .
..............................................................................................
|  2h 06m 30s 	Method 
|  2h 06m 30s 		20 start verses
		Genesis 10:2
|  2h 06m 30s 		         ----------> 1_Chronicles 1:5     confidende 100%
		Genesis 10:3
|  2h 06m 30s 		         ----------> 1_Chronicles 1:6     confidende  95%
		Genesis 10:4
|  2h 06

# Consolidate the continuous version
If you have run an update version called `_temp`, and all has went well
you can copy over the entire version (including its source and temp directories to `c`).
This will happen for all repos in the pipeline.

In [ ]:
#good = True
#good = False

if good:
    copyVersion(pipeline, '_temp', 'c')